In [3]:
##before continuing the following packages need to be runned
library(tidyverse)
library(repr)
library(tidymodels)
options(repr.matrix.max.rows = 6)
source('tests.R')
source('cleanup.R')

ERROR: Error: package or namespace load failed for ‘tidyverse’ in loadNamespace(j <- i[[1L]], c(lib.loc, .libPaths()), versionCheck = vI[[j]]):
 there is no package called ‘tidyr’


In [2]:
portfolio_data_raw <- read_csv("final_data.csv")|>
mutate(max_portfolio_return_name = as_factor(max_portfolio_return_name))
portfolio_data_raw

portfolio_data <- read_csv("final_data.csv")|>
mutate(max_portfolio_return_name = as_factor(max_portfolio_return_name))|>
select(-year, -max_portfolio_return_value)
portfolio_data

ERROR: Error in mutate(read_csv("final_data.csv"), max_portfolio_return_name = as_factor(max_portfolio_return_name)): could not find function "mutate"


In [4]:
#exloring our data:

num_obs <- nrow(portfolio_data)

portfolio_data |>
group_by(max_portfolio_return_name) |>
summarize(count = n(), percentage = n() / num_obs * 100)

#data seems to have too much "growth_portfolio" labels. Thus, rebalancing of classes could help here.

ERROR: Error in nrow(portfolio_data): object 'portfolio_data' not found


In [ ]:
set.seed(9999) #ensuring reproducibility
options(repr.plot.height = 5, repr.plot.width = 6)

portfolio_split <- initial_split(portfolio_data, prop = 0.75, strata = max_portfolio_return_name)
portfolio_train <- training(portfolio_split)
portfolio_test <- testing(portfolio_split) 

In [ ]:
knn_recipe <- recipe(max_portfolio_return_name ~ annual_inflation + real_gdp_growth_rate_percentage + annual_unemployment_rate, data = portfolio_train) |>
    step_center(all_predictors()) |>
    step_scale (all_predictors())

In [ ]:
knn_tune <- nearest_neighbor(weight_func = "rectangular", neighbors = tune()) |> 
set_engine("kknn") |>
set_mode("classification") 

In [ ]:
set.seed(9999) #ensuring reproducibility

knn_vfold <- vfold_cv(portfolio_train, v = 5, strata = max_portfolio_return_name)
gridvals <- tibble(neighbors = seq(from = 1, to = 15))

knn_results <- workflow() |>
  add_recipe(knn_recipe) |>
  add_model(knn_tune) |>
  tune_grid(resamples = knn_vfold, grid = gridvals) |>
  collect_metrics() 

accuracies <- knn_results |> 
       filter(.metric == "accuracy")

cross_val_plot <- ggplot(accuracies, aes(x = neighbors, y = mean))+
       geom_point() +
       geom_line() +
       labs(x = "Neighbors", y = "Accuracy Estimate")
cross_val_plot                                                                     

In [ ]:
# based on the plot,  we will pick K=8 because it the median neighbor between the equal-accuracy-range of K=4-12. choosing k=8 ensures that going a bit to the righ or left leaves the accuracy estimate intact
# We certainly believe that in order to enhance our model, more data should be collected for each variable. Unfortunatly, our data was restricted for data from 1997-2021. 

knn_spec <- nearest_neighbor(weight_func = "rectangular", neighbors = 8) |>
set_engine("kknn") |>
set_mode("classification")

knn_fit <- workflow() |>
add_recipe(knn_recipe) |>
add_model(knn_spec) |>
fit(data = portfolio_train)

In [ ]:
#testing model predictions with testing set

set.seed(9999) #ensuring reproducibility

test_predictions <- predict(knn_fit,portfolio_test) |>
bind_cols(portfolio_test)

knn_metrics <- test_predictions |>
metrics(truth = max_portfolio_return_name, estimate = .pred_class)
knn_metrics

#accuracy seems to improve a bit (from 70% up to ~71%) when applying model to testing dataset

In [ ]:
#This is the new observation to predict

new_economic_environment_1 <- tibble(annual_inflation = 1, real_gdp_growth_rate_percentage = 2.3, annual_unemployment_rate = 4)

portfolio_prediction <- predict(knn_fit, new_economic_environment_1)
portfolio_prediction

new_economic_environment_2 <- tibble(annual_inflation = 8, real_gdp_growth_rate_percentage = 2, annual_unemployment_rate = 4)

portfolio_prediction <- predict(knn_fit, new_economic_environment_2)
portfolio_prediction
    
    